In [ ]:
!pip install pandas-gbq==0.21.0

  Attempting uninstall: pandas-gbq
    Found existing installation: pandas-gbq 0.26.1
    Uninstalling pandas-gbq-0.26.1:
      Successfully uninstalled pandas-gbq-0.26.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.34.0 requires pandas-gbq>=0.26.0, but you have pandas-gbq 0.21.0 which is incompatible.


In [4]:
"""
ETL PROFISSIONAL: Vendas de E-commerce (Simulador Nível #9)
Autor: Eric Pimentel
"""

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from google.auth import default
from google.colab import auth
from google.cloud import bigquery
from googleapiclient.discovery import build
from google.cloud.bigquery import DatasetReference, TableReference
import datetime
import pandas_gbq
import gspread
import kagglehub
import os
import logging
import warnings
warnings.filterwarnings("ignore")

# Configurar logs
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


# 1. EXTRAÇÃO

def extract_data():
    """Extrai dados do Kaggle e garante o tipo correto para Data_Pedido."""
    try:
        logging.info(" Iniciando extração...")
        path = kagglehub.dataset_download("adarsh0806/influencer-merchandise-sales")
        files = os.listdir(path)

        for file in files:
            if file.endswith(".csv"):
                df = pd.read_csv(os.path.join(path, file), delimiter=',') # Removido parse_dates aqui
                # Conversão explícita para datetime com formato
                df['Order Date'] = pd.to_datetime(df['Order Date'], format='%Y-%m-%d')
                logging.info(" Dados extraídos com sucesso!")
                return df

        raise FileNotFoundError("Nenhum CSV encontrado.")

    except Exception as e:
        logging.error(f" Erro na extração: {e}")
        return None


# 2. TRANSFORMAÇÃO (COM REGRESSÃO LOGÍSTICA)

def transform_data(df):
    """Transforma dados e aplica modelo de previsão."""
    try:
        logging.info(" Iniciando transformação...")

        # ----- TRADUÇÃO DAS COLUNAS -----
        df = df.rename(columns={
            'Order ID': 'ID_Pedido',
            'Order Date': 'Data_Pedido',
            'Product Category': 'Categoria_Produto',
            'Buyer Gender': 'Genero_Comprador',
            'Buyer Age': 'Idade_Comprador',
            'Order Location': 'Localizacao_Pedido',
            'International Shipping': 'Envio_Internacional',
            'Sales Price': 'Preco_Venda',
            'Shipping Charges': 'Taxas_Envio',
            'Sales per Unit': 'Vendas_por_Unidade',
            'Quantity': 'Quantidade',
            'Total Sales': 'Vendas_Totais',
            'Rating': 'Avaliacao',
            'Review': 'Resenha'
        })

         # ----- CONVERSÃO DE TIPOS -----
        # Variáveis numéricas para float
        colunas_float = ['Preco_Venda', 'Taxas_Envio', 'Vendas_por_Unidade',
                        'Vendas_Totais', 'Avaliacao']
        for coluna in colunas_float:
            df[coluna] = df[coluna].astype(float)

        # Variáveis numéricas para int
        colunas_int = ['ID_Pedido', 'Idade_Comprador', 'Quantidade']
        for coluna in colunas_int:
            df[coluna] = df[coluna].astype(int)

        # Variáveis categóricas para string
        colunas_string = ['Categoria_Produto', 'Genero_Comprador',
                         'Localizacao_Pedido', 'Resenha']
        for coluna in colunas_string:
            df[coluna] = df[coluna].astype(str)

        # ----- LIMPEZA E ENGENHARIA DE FEATURES -----
        df = df.drop_duplicates(subset=['ID_Pedido'])
        df['Margem_Lucro'] = df['Vendas_Totais'] - (df['Preco_Venda'] * df['Quantidade'] + df['Taxas_Envio'])
        df['Faixa_Etaria'] = pd.cut(df['Idade_Comprador'], bins=[0, 18, 25, 35, 100], labels=['<18', '18-25', '26-35', '35+'])

        # ----- CORRIGIR COLUNA 'Envio_Internacional' -----
        df['Envio_Internacional'] = df['Envio_Internacional'].map({'Yes': 1, 'No': 0}).astype(int)

        if df['Envio_Internacional'].isnull().any():
            logging.warning("🚨 Valores inválidos encontrados em 'Envio_Internacional'. Substituindo por 0.")
            df['Envio_Internacional'] = df['Envio_Internacional'].fillna(0)

        # ----- PREPARAÇÃO PARA REGRESSÃO LOGÍSTICA -----
        df_mensal = df.groupby(pd.Grouper(key='Data_Pedido', freq='M')).agg({
            'Vendas_Totais': 'sum',
            'Avaliacao': 'mean',
            'Envio_Internacional': 'sum'
        }).reset_index()

        df_mensal['Target'] = (df_mensal['Vendas_Totais'].shift(-1) > df_mensal['Vendas_Totais']).astype(int)
        df_mensal = df_mensal.dropna()

        X = df_mensal[['Vendas_Totais', 'Avaliacao', 'Envio_Internacional']]
        y = df_mensal['Target']

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = LogisticRegression()
        model.fit(X_train, y_train)

        df_mensal['Previsao_Vendas'] = model.predict(X)
        logging.info(f" Modelo treinado! Acurácia: {accuracy_score(y_test, model.predict(X_test)):.2%}")

        df_mensal['Mes'] = df_mensal['Data_Pedido'].dt.strftime('%Y-%m')
        df['Mes'] = df['Data_Pedido'].dt.strftime('%Y-%m')
        df = pd.merge(df, df_mensal[['Mes', 'Previsao_Vendas']], on='Mes', how='left')

        logging.info(" Dados transformados e previsões geradas!")
        return df

    except Exception as e:
        logging.error(f" Erro na transformação: {e}")
        return None


# 3. CARREGAMENTO (ATUALIZADO COM PREVISÕES)

def load_data(df):
    """Carrega dados para Google Sheets e BigQuery."""
    try:
        logging.info(" Iniciando carregamento...")
        auth.authenticate_user()
        credentials, _ = default()
        gc = gspread.authorize(credentials)

        # Converter a coluna 'Data_Pedido' para string
        df['Data_Pedido'] = df['Data_Pedido'].dt.strftime('%Y-%m-%d %H:%M:%S')

        # Google Sheets
        spreadsheet = gc.create('Vendas_Tratadas_Influenciadorv2.0')
        spreadsheet.share(None, perm_type='anyone', role='writer')
        worksheet = spreadsheet.get_worksheet(0)
        worksheet.update([df.columns.tolist()] + df.values.tolist())

        print(f"URL da planilha: {spreadsheet.url}")

        # BigQuery - ESQUEMA DEFINIDO
        project_id = 'projeto-vendas-influenciador'
        dataset_id = 'projeto_vendas'
        table_id = 'vendas_tratadas_v2'

        # CONVERSÃO EXPLÍCITA PARA DATETIME64[NS] ANTES DO CARREGAMENTO
        df['Data_Pedido'] = pd.to_datetime(df['Data_Pedido']).dt.tz_localize(None) # Correção crucial

        # Converter 'Avaliacao' para numérico (opcional, mas recomendado)
        df['Avaliacao'] = pd.to_numeric(df['Avaliacao'], errors='coerce')

        # Definir o esquema da tabela (opcional, se já definiu no BigQuery)
        schema = [
            {'name': 'ID_Pedido', 'type': 'INTEGER'},
            {'name': 'Data_Pedido', 'type': 'DATE'},  # Tipo DATE
            # ... outras colunas ...
            {'name': 'Avaliacao', 'type': 'FLOAT'}, # Avaliacao como FLOAT
            {'name': 'Mes', 'type': 'STRING'},
            {'name': 'Previsao_Vendas', 'type': 'INTEGER'}
        ]

        # Configurar cliente do BigQuery
        client = bigquery.Client(project=project_id)

        # Criar dataset se não existir
        dataset_ref = client.dataset(dataset_id)
        try:
            client.get_dataset(dataset_ref)
        except Exception:
            dataset = bigquery.Dataset(dataset_ref)
            dataset.location = "US"  # Especificar localização
            client.create_dataset(dataset, exists_ok=True)

        # Carregar para o BigQuery com esquema definido
        pandas_gbq.to_gbq(
            df,
            destination_table=f"{dataset_id}.{table_id}",
            project_id=project_id,
            if_exists='replace',
            table_schema=schema
        )
        logging.info(" Dados no BigQuery: Prontos para SQL avançado!")
        print(f"URL do projeto no BigQuery: https://console.cloud.google.com/bigquery?project={project_id}&p={project_id}&d={dataset_id}&t={table_id}&page=table")

        return True

    except Exception as e:
        logging.error(f" Erro no carregamento: {e}")
        return False


# 4. EXECUÇÃO PRINCIPAL

if __name__ == "__main__":
    dados_brutos = extract_data()
    if dados_brutos is not None:
        dados_tratados = transform_data(dados_brutos)
        if dados_tratados is not None:
            sucesso = load_data(dados_tratados)
            if sucesso:
                logging.info(" Missão cumprida! Previsões no Looker!")
            else:
                logging.warning(" Concluído com erros parciais.")

URL da planilha: https://docs.google.com/spreadsheets/d/1kPfKIkysuCrS4gRfwS841mPtFDeRQG2zpAjbmPAffWA


100%|██████████| 1/1 [00:00<00:00, 756.14it/s]

URL do projeto no BigQuery: https://console.cloud.google.com/bigquery?project=projeto-vendas-influenciador&p=projeto-vendas-influenciador&d=projeto_vendas&t=vendas_tratadas_v2&page=table
